In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

class PrcRefinanciados():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d')
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
  
  def actualizarVan(self):
    
    #Obteniendo clientes refinanciados
    select_Refinanciados = """SELECT REFINANCIAMIENTO_VCTO, FECHA_PROCESO, CLIENTE_DEUDOR_PRODUCTO, FECHA_VENCIMIENTO, CAPITAL, INTERES, VAN_SIN_INVERSION FROM
                              (
                              SELECT REFINANCIAMIENTO_VCTO, FECHA_PROCESO, CLIENTE_DEUDOR_PRODUCTO, CONCEPTO_SALDO, CAPITAL, INTERES, FECHA_VENCIMIENTO, FECHA_INICIO, FECHA_FIN, TASA_DIARIA, CUOTA, DATEDIFF(D, FECHA_VENCIMIENTO, FECHA_FIN) AS DIFERENCIA_DIAS, (CUOTA/POWER((1 + TASA_DIARIA), DATEDIFF(D, FECHA_VENCIMIENTO, FECHA_FIN))) AS VAN_SIN_INVERSION FROM
                              (
                              SELECT REFINANCIAMIENTO_VCTO, FECHA_PROCESO, CLIENTE_DEUDOR_PRODUCTO, CONCEPTO_SALDO, CAPITAL, INTERES, FECHA_VENCIMIENTO, CONVERT(VARCHAR(25), DATEADD(DD, - (DAY(FECHA_PROCESO) - 1), FECHA_PROCESO), 23) AS FECHA_INICIO, EOMONTH(FECHA_PROCESO) AS FECHA_FIN, TASA, (POWER((1 + (TASA/100.0)), (1.0/360.0)) - 1) AS TASA_DIARIA, (CAPITAL + INTERES) AS CUOTA FROM
                              (
                              SELECT RVD.REFINANCIAMIENTO_VCTO, R.FECHA_PROCESO, R.CLIENTE_DEUDOR_PRODUCTO, RV.CONCEPTO_SALDO, RVD.CAPITAL, RVD.INTERES, RVD.FECHA_VENCIMIENTO, RT.TASA
                              FROM REFINANCIAMIENTO R JOIN REFINANCIAMIENTO_VCTO RV
                              ON R.ID = RV.REFINANCIAMIENTO
                              JOIN REFINANCIAMIENTO_VCTO_DET RVD
                              ON RV.ID = RVD.REFINANCIAMIENTO_VCTO
                              JOIN REFINANCIAMIENTO_TASA RT
                              ON R.FECHA_PROCESO = RT.FECHA
                              AND RV.CONCEPTO_SALDO = RT.CONCEPTO_SALDO
                              WHERE LEFT(FECHA_PASE_REF, 7) = LEFT('""" + self.fecha_proceso_str + """', 7)
                              ) A
                              ) B
                              ) C"""
    sql_Refinanciados = pd.read_sql_query(select_Refinanciados, self.connection)
    df_Refinanciados = pd.DataFrame(sql_Refinanciados)
    df_Refinanciados['FECHA_ACTUALIZACION'] = self.fecha_actualizacion_str
    df_Refinanciados = df_Refinanciados[['VAN_SIN_INVERSION', 'FECHA_ACTUALIZACION','REFINANCIAMIENTO_VCTO', 'FECHA_PROCESO']]
    
    #Actualizando van de clientes refinanciados  
    update_refinanciamiento_vcto_det = "UPDATE [DBO].REFINANCIAMIENTO_VCTO_DET SET VAN_SIN_INVERSION = ?, FECHA_ACTUALIZACION = ? WHERE REFINANCIAMIENTO_VCTO = ? AND FECHA_PROCESO = ?"
    self.cursor.fast_executemany = True
    values = df_Refinanciados.values.tolist()
    if values:
      self.cursor.executemany(update_refinanciamiento_vcto_det, values)
      self.connection.commit()    
    self.cursor.close()    